pip install datasets -q

In [ ]:
pip install rouge_score -q

In [1]:
import transformers
print(transformers.__version__)

4.15.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

In [105]:
pip install huggingface_hub -q

You should consider upgrading via the '/home/ccal0507/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a summarization task. We will use the [XSum dataset](https://arxiv.org/pdf/1808.08745.pdf) (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.

![Widget inference on a summarization task](images/summarization.png)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [2]:
model_checkpoint = "t5-small"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [3]:
from datasets import load_metric

#metric = load_metric("bertscore")
metric = load_metric("rouge")

In [4]:
from datasets import load_dataset

In [5]:
import pandas as pd

In [6]:
#datasets = load_dataset('csv', data_files='cleaned_lyrics.csv')

#seventies_data = dataset[(dataset['year'] > 1969) & (dataset['year'] < 1980)].reset_index()


dataset = pd.read_csv('cleaned_lyrics.csv')
dataset = dataset.query("description != '?'")
dataset = dataset.rename(columns={'lyrics':'document', 'description':'summary'})
dataset['document'] = dataset['document'].str.replace('\n', ' ')
dataset

,artist,title,document,summary,release date,year
0,ray charles,hit the road jack,hit the road jack and doncha come back no mor...,This tongue and cheek verbal duel of a couple ...,August 1961,1961
1,ray charles,georgia on my mind,georgia georgia the whole day through (the wh...,Written by Hoagy Carmichael and Stuart Gorrell...,September 1960,1960
2,ray charles,i’ve got a woman,well i got a woman way over town thats good ...,Ray Charles released “I’ve Got a Woman” as a s...,December 1954,1954
8,ghetto brothers,girl from the mountain,nanana nanananana (nana) nanana nanananana (n...,The only track on Power-Fuerza not written by ...,"January 1, 1971",1971
11,ghetto brothers,viva puerto rico libre,buenas amados hermanos compañeros y compañeras...,Benji Melendez’s family moved from Puerto Rico...,"January 1, 1972",1972
...,...,...,...,...,...,...
368,raveena,temptation,ahahah ahahah miss temptation i dont think yo...,"In “Temptation”, Raveena opens up about her bi...","October 23, 2018",2018
369,the notorious b.i.g.,juicy,"(""fuck all you hoes"" get a grip motherfucker)...",“Juicy” is the first single from Big’s debut a...,"August 9, 1994",1994
370,the notorious b.i.g.,big poppa,uh uh check it out (yeah) uh junior mafia uh ...,“Big Poppa” was The Notorious B.I.G.’s first t...,"February 20, 1995",1995
371,the notorious b.i.g.,suicidal thoughts,hello aw shit nigga the fuck time is it man o...,In this final track off of The Notorious B.I.G...,"September 13, 1994",1994


In [7]:
from datasets import Dataset

In [8]:
datasets = Dataset.from_pandas(dataset, split='train[:30%]').train_test_split()

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__'],
        num_rows: 234
    })
    test: Dataset({
        features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__'],
        num_rows: 79
    })
})

In [212]:
datasets['test']['document'][:2]

[' woahohohoh woahohohoh woahohohoh woahohohoh woahohohoh woahohohoh  im crazy flowing over with ideas a thousand ways to woo a lover so sincere love and hate what a beautiful combination sending shivers up and down my spine  for every casanova that appears my sense of hesitation disappears love and hate what a beautiful combination sending shivers up and down my spine  and the lovers that you sent for me didnt come with any satisfaction guarantee so i return them to the sender and the note attached will read how i love to hate you i love to hate you i love to hate you i love to hate you  oh you really still expect me to believe every single letter i receive sorry what a shameful situation sending shivers up and down my spine  ooh i like to read murder mystery i like to know the killer isnt me love and hate what a beautiful combination sending shivers make me quiver feel it slither up and down my spine  and the lovers that you sent for me didnt come with any satisfaction guarantee so i

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [10]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [13]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
#metric.compute(predictions=fake_preds, references=fake_labels, lang="en")
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [12]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [13]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [14]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [15]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__'],
        num_rows: 234
    })
    test: Dataset({
        features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__'],
        num_rows: 79
    })
})

In [17]:
datasets['train']

Dataset({
    features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__'],
    num_rows: 234
})

In [18]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True) #, return_tensors="tf") #, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding=True) #, return_tensors="tf") #, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
#tokenized_datasets = seventies_data.apply(preprocess_function, axis=1)

tokenized_datasets = datasets.map(preprocess_function , batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 234
    })
    test: Dataset({
        features: ['artist', 'title', 'document', 'summary', 'release date', 'year', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 79
    })
})

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

# Fine Tuning Model with TensorFlow

In [21]:
from transformers import DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM, create_optimizer, AdamWeightDecay

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model =  TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

#columns_to_return = ['input_ids', 'labels', 'attention_mask']
#tokenized_datasets.set_format(type='tf', columns=columns_to_return)

In [23]:
tf_train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'labels'],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator)

tf_test_set = tokenized_datasets["test"].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'labels'],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator)

In [24]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [25]:
import numpy as np
import nltk


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [26]:
from keras_callbacks import KerasMetricCallback

In [27]:
metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=tf_test_set, predict_with_generate=True
)

In [28]:
model.fit(
    tf_train_set, validation_data=tf_test_set, epochs=1, callbacks=[metric_callback]
)

29/29 [==============================] - 249s 8s/step - loss: 6.7569 - val_loss: 5.6025


In [49]:
tokenized_datasets['test']['document'][0]

' woahohohoh woahohohoh woahohohoh woahohohoh woahohohoh woahohohoh  im crazy flowing over with ideas a thousand ways to woo a lover so sincere love and hate what a beautiful combination sending shivers up and down my spine  for every casanova that appears my sense of hesitation disappears love and hate what a beautiful combination sending shivers up and down my spine  and the lovers that you sent for me didnt come with any satisfaction guarantee so i return them to the sender and the note attached will read how i love to hate you i love to hate you i love to hate you i love to hate you  oh you really still expect me to believe every single letter i receive sorry what a shameful situation sending shivers up and down my spine  ooh i like to read murder mystery i like to know the killer isnt me love and hate what a beautiful combination sending shivers make me quiver feel it slither up and down my spine  and the lovers that you sent for me didnt come with any satisfaction guarantee so i 

In [124]:
import tensorflow as tf

In [84]:
#example = tf.convert_to_tensor(tokenized_datasets['test']['document'][0])
example = tokenized_datasets['test']['document'][0]
example

' woahohohoh woahohohoh woahohohoh woahohohoh woahohohoh woahohohoh  im crazy flowing over with ideas a thousand ways to woo a lover so sincere love and hate what a beautiful combination sending shivers up and down my spine  for every casanova that appears my sense of hesitation disappears love and hate what a beautiful combination sending shivers up and down my spine  and the lovers that you sent for me didnt come with any satisfaction guarantee so i return them to the sender and the note attached will read how i love to hate you i love to hate you i love to hate you i love to hate you  oh you really still expect me to believe every single letter i receive sorry what a shameful situation sending shivers up and down my spine  ooh i like to read murder mystery i like to know the killer isnt me love and hate what a beautiful combination sending shivers make me quiver feel it slither up and down my spine  and the lovers that you sent for me didnt come with any satisfaction guarantee so i 

In [85]:
example = tokenizer(example, max_length=8, truncation=True, padding=True)

In [86]:
example

{'input_ids': [2275, 9, 107, 32, 107, 32, 107, 32, 107, 2275, 9, 107, 32, 107, 32, 107, 32, 107, 2275, 9, 107, 32, 107, 32, 107, 32, 107, 2275, 9, 107, 32, 107, 32, 107, 32, 107, 2275, 9, 107, 32, 107, 32, 107, 32, 107, 2275, 9, 107, 32, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [81]:
tf.convert_to_tensor(example['input_ids'])

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([2275,    9,  107,   32,  107,   32,  107,    1], dtype=int32)>

In [74]:
model.generate(input_ids=tf.convert_to_tensor(example['input_ids']))

AssertionError: Input prompt should be of shape (batch_size, sequence length).

# Fine-tuning the model with PyTorch

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [87]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [88]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [89]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [90]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [181]:
columns_to_return = ['input_ids', 'labels', 'attention_mask']
tokenized_datasets.set_format(type='torch', columns=columns_to_return)

In [91]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    #train_dataset=train_data,
    #eval_dataset=dev_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [92]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: release date, document, title, summary, year, artist, __index_level_0__.
***** Running training *****
  Num examples = 234
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 60


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.810317,16.820000,2.323700,13.863400,16.224800,19.000000
2,No log,3.662947,14.629200,3.118200,12.252800,13.454600,19.000000
3,No log,3.633828,10.234100,1.962000,8.389600,9.083600,19.000000
4,No log,3.676234,14.274700,3.098200,11.565300,12.883400,19.000000


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: release date, document, title, summary, year, artist, __index_level_0__.
***** Running Evaluation *****
  Num examples = 79
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: release date, document, title, summary, year, artist, __index_level_0__.
***** Running Evaluation *****
  Num examples = 79
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: release date, document, title, summary, year, artist, __index_level_0__.
***** Running Evaluation *****
  Num examples = 79
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored

TrainOutput(global_step=60, training_loss=3.6088185628255207, metrics={'train_runtime': 999.7291, 'train_samples_per_second': 0.936, 'train_steps_per_second': 0.06, 'total_flos': 253359852355584.0, 'train_loss': 3.6088185628255207, 'epoch': 4.0})

In [96]:
type(trainer)

transformers.trainer_seq2seq.Seq2SeqTrainer

In [201]:
sample = tokenized_datasets['train'].shuffle().select(list(range(36)))

In [202]:
sample['input_ids']

tensor([[21603,    10,    25,  ...,     0,     0,     0],
        [21603,    10,  1234,  ...,     0,     0,     0],
        [21603,    10, 10556,  ...,     0,     0,     0],
        ...,
        [21603,    10,     3,  ...,     0,     0,     0],
        [21603,    10,  4743,  ...,     0,     0,     0],
        [21603,    10,  8524,  ...,     0,     0,     0]])

In [203]:
sample_encodings =  model.generate(sample['input_ids'], 
                                   attention_mask=sample["attention_mask"])

In [204]:
sample_decodings = tokenizer.batch_decode(sample_encodings, skip_special_tokens=True)

In [205]:
sample_decodings

['“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, released as the second single from',
 '“Sold Out” is the first single from the album, relea